In [2]:
import pandas as pd
import cv2
import os
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

# import tensorflow as tf
import keras
import keras.backend as K
# from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
# import keras_retinanet
import keras_retinanet.bin.train
from sklearn.utils import shuffle 
from tqdm import tqdm_notebook as tqdm

In [3]:
json_file = open('/home/jovyan/model.json', 'r')
model_json = json_file.read()
json_file.close()
model = keras.models.model_from_json(model_json)

In [4]:
test_data = pd.read_json('../test_data_with_obj_id.json')
test_data = test_data.sort_index()
retinanet_test_data = pd.read_json('retinanet_test_data.json')
retinanet_test_data = retinanet_test_data.sort_index()

In [5]:
img_input = keras.layers.Input(shape=(None,None,3),name='img_input')
obj_boxes = keras.layers.Input(shape=(None,4),name='obj_boxes')
obj_classes = keras.layers.Input(shape=(None,),name='obj_classes')
human_boxes = keras.layers.Input(shape=(None,4),name='human_boxes')

In [6]:
test_data.head()

,action_no,human_bbox,invisible,name,obj_id,obj_list,object_bbox,pair_no,size
0,[246],"[[320, 359, 306, 349], [270, 311, 303, 350]]",[0],HICO_test2015_00000001.jpg,[13],[bench],"[[148, 376, 345, 414]]","[[1, 1], [2, 1]]","[640, 427, 3]"
1,"[132, 140, 144]","[[226, 340, 18, 210], [230, 356, 19, 212], [23...","[0, 0, 0]",HICO_test2015_00000002.jpg,[17],[horse],"[[174, 393, 65, 440], [186, 392, 62, 438], [17...","[[1, 1], [1, 1], [1, 1]]","[640, 461, 3]"
2,[39],"[[302, 322, 182, 236]]",[0],HICO_test2015_00000003.jpg,[8],[boat],"[[234, 444, 85, 298]]","[[1, 1]]","[640, 425, 3]"
3,"[154, 155]","[[411, 481, 83, 186], [464, 547, 118, 221], [3...","[0, 0]",HICO_test2015_00000004.jpg,[3],[motorcycle],"[[338, 574, 109, 247], [110, 501, 183, 364], [...","[[1, 1], [2, 1], [3, 2], [4, 2], [1, 1], [2, 2]]","[640, 427, 1]"
4,"[209, 213]","[[5, 409, 62, 638], [2, 415, 60, 637]]","[0, 0]",HICO_test2015_00000005.jpg,[24],[backpack],"[[197, 466, 167, 638], [327, 470, 165, 524]]","[[1, 1], [1, 1]]","[480, 640, 3]"


In [7]:
retinanet_test_data.head()

,human_boxes,human_boxes_scale,human_classes,human_scores,obj_boxes,obj_boxes_scale,obj_classes,obj_scores
0,"[[499.9312438965, 570.3104858398, 585.98291015...","[[266, 304, 312, 348], [312, 305, 355, 347]]","[0, 0]","[0.8959407210000001, 0.8261852264]","[[499.9312438965, 570.3104858398, 585.98291015...","[[266, 304, 312, 348], [312, 305, 355, 347]]","[0, 0]","[0.8959407210000001, 0.8261852264]"
1,"[[839.0675048828, 258.8868713379, 863.58618164...","[[447, 149, 460, 190], [164, 137, 200, 220], [...","[0, 0, 0, 0, 0, 0, 0]","[0.2363488078, 0.9441347718, 0.912345409400000...","[[839.0675048828, 258.8868713379, 863.58618164...","[[447, 149, 460, 190], [193, 159, 218, 185], [...","[0, 2, 26, 26, 0, 0, 0, 16, 0, 17, 0, 17, 17, 0]","[0.2363488078, 0.21155761180000002, 0.20389854..."
2,"[[622.6060791016, 326.1147460938, 648.74401855...","[[332, 173, 345, 197], [304, 176, 321, 196], [...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.2099235505, 0.2889916003, 0.3744371235, 0.4...","[[622.6060791016, 326.1147460938, 648.74401855...","[[332, 173, 345, 197], [304, 176, 321, 196], [...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, ...","[0.2099235505, 0.2889916003, 0.3744371235, 0.4..."
3,"[[769.3322753906, 162.2735290527, 889.46887207...","[[410, 86, 474, 165], [404, 86, 479, 222], [42...","[0, 0, 0, 0, 0, 0, 0]","[0.2236517966, 0.2772318125, 0.8223848939, 0.9...","[[769.3322753906, 162.2735290527, 889.46887207...","[[410, 86, 474, 165], [404, 86, 479, 222], [42...","[0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 2, 1, 3, 1, 3]","[0.2236517966, 0.2772318125, 0.8223848939, 0.9..."
4,"[[17.4734802246, 483.500213623, 540.1184082031...","[[6, 386, 216, 624], [10, 59, 425, 622]]","[0, 0]","[0.2655027807, 0.9683949351000001]","[[946.7169189453, 689.7432861328, 1017.5393066...","[[378, 551, 407, 636], [324, 174, 462, 447], [...","[56, 59, 0, 63, 0]","[0.2003484666, 0.2143364251, 0.2655027807, 0.2..."


In [8]:
np.max(retinanet_test_data.human_scores[1])

0.94413477180000005

In [9]:
def data_gen(retinanet_test_data,test_data,id_):
    img_stack = np.array([]).reshape(0,800,1200,3)
    human_stack = np.array([]).reshape(0,1,4)
    object_stack = np.array([]).reshape(0,1,4)
    object_class_stack = np.array([]).reshape(0,1)
            #image
    img = cv2.imread(os.path.join('/home/jovyan/projectdata/cht01/hico_20160224_det/images/test2015/',test_data['name'][id_]))
    img = cv2.resize(img, (1200,800))
    img = img/255
    img = np.expand_dims(img,axis=0)
            
            #human object bbox pair & object class
    for i in range(len(retinanet_test_data.human_boxes[id_])):
        if (np.max(retinanet_test_data.human_scores[id_]) >0.5) & (retinanet_test_data.human_scores[id_][i]<0.5):
            continue
        ho_pair_h = retinanet_test_data.human_boxes[id_][i]
        ho_pair_h = np.expand_dims(ho_pair_h,axis=0)
        ho_pair_h = np.expand_dims(ho_pair_h,axis=0)
        for j in range(len(retinanet_test_data.obj_boxes[id_])):
            if (np.max(retinanet_test_data.obj_scores[id_]) >0.5) & (retinanet_test_data.obj_scores[id_][j]<0.5):
                continue
            ho_pair_o = retinanet_test_data.obj_boxes[id_][j]
            object_class = retinanet_test_data.obj_classes[id_][j]
            ho_pair_o = np.expand_dims(ho_pair_o,axis=0)
            ho_pair_o = np.expand_dims(ho_pair_o,axis=0)                    
            object_class = np.expand_dims(object_class,axis=0)
                    
            img_stack = np.row_stack([img_stack,img])
            human_stack = np.row_stack([human_stack,ho_pair_h])
            object_stack = np.row_stack([object_stack,ho_pair_o])
            object_class_stack = np.row_stack([object_class_stack,object_class])
                
    test_batch = [img_stack,human_stack,object_stack,object_class_stack]
    return test_batch

In [14]:
# model_multigpu = keras.utils.multi_gpu_model(model,gpus=2)

In [15]:
pred_all = []
for i in tqdm(range(test_data.shape[0])):
    test_one_img = data_gen(retinanet_test_data,test_data,i)
    pred = model.predict(test_one_img)
    pred_all.append(pred)

HBox(children=(IntProgress(value=0, max=9658), HTML(value='')))

KeyboardInterrupt: 